In [15]:
from pandas import read_csv
from datetime import datetime
import matplotlib as mpl
from matplotlib import pylab as plt
import matplotlib.dates as mdates
import pandas as pd
import collections
import seaborn as sns
import numpy as np
import tensorflow as tf

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
import pickle

Using TensorFlow backend.


In [16]:
# load data
currencies = ['EXR.D.AUD.EUR.SP00.A', 'EXR.D.BRL.EUR.SP00.A', 'EXR.D.CAD.EUR.SP00.A', 'EXR.D.CNY.EUR.SP00.A', 'EXR.D.GBP.EUR.SP00.A',
              'EXR.D.HKD.EUR.SP00.A', 'EXR.D.IDR.EUR.SP00.A', 'EXR.D.INR.EUR.SP00.A', 'EXR.D.JPY.EUR.SP00.A', 'EXR.D.MYR.EUR.SP00.A']
def builData(curr_key):
    
    print('Loading data and parsing date')
    
    def parse(x):
        return datetime.strptime(x, '%Y-%m-%d')

    df = read_csv('inr.csv', parse_dates=['date'], date_parser=parse)
    print('Data loaded succesfully ')

    # dataset.index.name = 'date'

    # dataset.drop('Unnamed: 0', axis=1, inplace=True)

    # mark all NA values with 0
    
    for curr in df.columns:
        if df[curr][3] == 0:
            df.drop(curr, axis=1, inplace=True)
            print('Deleted currency: ', curr)
        elif 'Unnamed' in curr:
            df.drop(curr, axis=1, inplace=True)
            print('Deleted currency: ', curr)
        
        elif curr != 'date' and curr in curr_key:
            
            print('Reading currency: ', curr)
            df[curr].fillna(0, inplace=True)
            df[curr] = df[curr].replace('-', 0)

            i = 0
            for v in df[curr]:                

                df[curr][i] = float(df[curr][i])
                if i < len(df[curr])-1 and df[curr][i] == 0:
                    df[curr][i] = float(df[curr][i+1])        
                i += 1

            print('Selected currency: ', curr)
        elif curr != 'date':
            df.drop(curr, axis=1, inplace=True)  
            print('Deleted currency: ', curr)


    df.info()
    print(df.head(5))
    curr_key = df.columns
    return df, curr_key[1:]

In [17]:
df, curr_key = builData(currencies)
df.to_pickle('currency_day')

Loading data and parsing date
Data loaded succesfully 
Deleted currency:  Unnamed: 0
Reading currency:  EXR.D.AUD.EUR.SP00.A


/home/jojo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/jojo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


KeyboardInterrupt: 

In [24]:
def prepareData(df):
    #creating train and test sets
    start_date = '1999-01-04'
    start = 365 * 15 #2015
    # df.columns.drop(['Unnamed: 0', 'date'])

    dataset = df.values

    dataset = np.array(sorted(dataset, key=lambda p: p[0], reverse=False))
    dataset = dataset[-start:, 1:].astype(np.float64)
    print(dataset.shape)
    n_curr = len(df.columns) - 1

    train_size = int(dataset.shape[0]*0.8)

    train = dataset[:train_size, :]
    valid = dataset[train_size:, :]
    #converting dataset into x_train and y_train
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(train)
    scaled_data_test = scaler.fit_transform(valid)

    x_train, y_train = [], []
    x_test, y_test = [], []

    lookback = 60
    for i in range(lookback,len(train)):
        x_train.append(scaled_data[i-lookback:i,:])
        y_train.append(scaled_data[i,:])
    for i in range(lookback,len(valid)):
        x_test.append(scaled_data_test[i-lookback:i,:])
        y_test.append(scaled_data_test[i,:])

    x_train, y_train = np.array(x_train), np.array(y_train)
    x_test, y_test = np.array(x_test), np.array(y_test)

    x_train = np.reshape(x_train, (x_train.shape[0],x_train.shape[1],n_curr))
    x_test = np.reshape(x_test, (x_test.shape[0],x_test.shape[1],n_curr))
    print(x_train.shape)
    print(y_train.shape)
    return x_train, y_train, x_test, y_test, scaler

In [2]:
def buildModel(x_train, y_train, curr_key):
    i = 0
    horizon = 50
    for curr in curr_key:
        # create and fit the LSTM network
        model = Sequential()
        model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1],1)))
        model.add(LSTM(units=horizon, dropout=0.2, recurrent_dropout=0.2))
        model.add(Dense(1))
        X = x_train[:, :, i].reshape(x_train.shape[0], x_train.shape[1], 1)
        y = y_train[:, i]

        model.compile(loss='mean_squared_error', optimizer='adam')
        history = model.fit(X, y, epochs=1, batch_size=1, verbose=1)
        filename = curr + '_model.sav'
        pickle.dump(model, open(filename, 'wb'))
        print('Model created :', curr)

        i += 1
        model = 0
        
    print('Total created models: ', i)
    

In [3]:
def loadModel(curr_key):
    i = 0
    models = []
    for curr in curr_key:
        # load the model from disk
        filename = curr + '_model.sav'
        loaded_model = pickle.load(open(filename, 'rb'))
        models.append(loaded_model)
        print('Loaded model: ', filename)
    return models

In [4]:
def predict(x_test, models):
    predict_arr = np.zeros((x_test.shape[0], len(models)))

    i = 0
    for model in models:
        X = x_test[:, :, i].reshape(x_test.shape[0], x_test.shape[1], 1)
        
        prediction = model.predict(X)
        predict_arr[:, i] = prediction.reshape(prediction.shape[0],)
        i += 1
    return predict_arr

In [5]:
def getMSE(x_train, y_train, models, scaler):
    i = 0
    mse_arr = []
    ps = 0*y_train
    for model in models:
        X = x_train[:, :, i].reshape(x_train.shape[0], x_train.shape[1], 1)
        p = model.predict(X)
        ps[:, i] = p.reshape(p.shape[0],)
        i += 1

    y_train_inv = scaler.inverse_transform(y_train)
    predict = scaler.inverse_transform(ps)
    
    for j in range(len(models)):
        mse = mean_squared_error(y_train_inv[:, j], predict[:, j])
        mse_arr.append(mse)
    return mse_arr

In [6]:
df = pd.read_pickle('currency_day')
df.head(5)

NameError: name 'pd' is not defined

In [ ]:
x_train, y_train, x_test, y_test, scaler = prepareData(df)

In [7]:
curr_key = currencies
print(curr_key)
buildModel(x_train, y_train, curr_key)

NameError: name 'currencies' is not defined

In [ ]:
print(curr_key)

In [ ]:
models = loadModel(curr_key)

In [8]:
predict_arr = predict(x_test, models)

NameError: name 'x_test' is not defined

In [ ]:
y_test_inv = scaler.inverse_transform(y_test)
predict = scaler.inverse_transform(predict_arr)

mse_arr = getMSE(x_train, y_train, models, scaler)

horizon = 100
for i in range(len(curr_key)):
    xs = np.arange(int(horizon))
    fig = plt.figure(figsize=(12, 6))
    ax = fig.add_subplot(1, 1, 1)

    ax.plot(xs, y_test_inv[-horizon:, i], label='Real ' + curr_key[i], lw=2, color='blue')
    ax.plot(xs, predict[-horizon:, i], label='Forecast ' + curr_key[i], lw=2, color='red')

    rmse = mse_arr[i]

    rmse_max = [float(p) + float(rmse)/2 for p in predict[:, i]]
    rmse_min = [float(p) - float(rmse)/2 for p in predict[:, i]]

    ax.fill_between(xs, rmse_max[-horizon:], rmse_min[-horizon:], color='red', alpha=0.2)
    plt.show()

In [9]:
def getActions(y, V, Cash, T):
    n = V.shape[1]
    t_cur = T
    t_prev = t_cur - 1
    t_prev2 = t_prev - 1
    t_next = t_cur + 1
    
    X_eur = Cash[t_cur, -1]
    X_new = X_eur
    A = []
    
    for c in range(n):
        action = ''
        X = 0

        d_prev = y[t_prev, c] - y[t_prev2, c]
        d_cur = y[t_cur, c] - y[t_prev, c]
        d_next = V[t_next, c] - y[t_cur, c]

        if d_prev > 0 and d_cur > 0 and d_next < 0:
            action = 'BUY'

        elif d_prev > 0 and d_cur < 0 and d_next < 0:
            action = 'BUY'

        elif d_prev < 0 and d_cur < 0 and d_next > 0:
            action = 'SELL'

        elif d_prev < 0 and d_cur > 0 and d_next > 0:
            action = 'SELL'
        A.append([action, X])
    
    return A

In [10]:
horizon=100

V = predict[-horizon-1:, :]
y = y_test_inv[-horizon-1:, :]
V[0, :] = y_test_inv[-horizon-1, :]
n = V.shape[1]
initial_cash = 1000

Cash = np.zeros((horizon+1, n + 1))
Cash[0, -1] = initial_cash
Cash[1, -1] = initial_cash
print('n ', n)

for t in range(2, horizon-1):
    A = getActions(y, V, Cash, t)
    Cash[t, :] = Cash[t-1, :]
    b = 0
    c = 0
    
    print('Jour ', t)
    for a in A:        
        if a[0] == 'SELL' and Cash[t, c] > 0:
            print(str(a[0]) + ' ' + str(Cash[t, c]) + ' ' + str(curr_key[c]) + ' rate :' + str(y[t, c]))
            
            Cash[t, -1] = Cash[t, -1] + Cash[t, c]/y[t, c]
            Cash[t, c] = Cash[t, c] -  Cash[t, c]
            
            
            print('Euro after sell ', Cash[t, -1])
        if a[0] == 'BUY':
            b += 1
        c += 1
          
    c = 0    
    for a in A:
            
        if a[0] == 'BUY' and Cash[t, -1] > 0:
            
            print(str(a[0]) + ' ' + str(Cash[t, -1]*y[t, c]/b) + ' ' + str(curr_key[c]) + ' rate: ' + str(y[t, c]/b))
            
            Cash[t, c] = Cash[t, c] + Cash[t, -1]*y[t, c]/b
            Cash[t, -1] = Cash[t, -1] - Cash[t, -1]/b
            
            print('Euro after buy ', Cash[t, -1])            
        
        c += 1
    
    print('Actions end of day: ', A)
    print('Accounts status: ', Cash[t, :])
    last_day = t

final_cash = Cash[last_day, -1]
print(final_cash)
for i in range(n):
    if y[last_day, i] != 0:
        final_cash += Cash[last_day, i]/y[last_day, i]
        initial_cash += Cash[1, i]/y[1, i]

d_growth = (final_cash/initial_cash)**(1/(last_day - 2)) - 1
                 
print('Initial cash value (eur): ', initial_cash)
print('Final cash value (eur): ', final_cash)
print('Daily growth: ', d_growth, '%')
print('Win: ', final_cash - initial_cash)

TypeError: 'function' object is not subscriptable

In [11]:
def cost(M, P):
    return -g(M, P)**2

In [12]:
#### Note pour plus tard: dg a 30,4 dimension mais g a une
## Définir fonction gain avec deltas
def regGain(Delta, P):
    n, m = Delta.shape[0], Delta.shape[1]
    g = np.zeros((1, m*m))
    for k in range(1, n):
        for c in range(m*m):
            
            g[k, 0] = 1 - P[k, c1]/P[k, c2]
    return g


In [13]:
def regCost(Delta, P):
    return regGain(Delta, P)

In [14]:
# Our initinal guess
horizon=30
P = predict[-horizon:, :]
print(P.shape)
alpha = .1
horizon=30

epoch = 10000

grad = P
print(grad.shape)
m = Delta.shape[1]
Delta = np.random.rand(30, m*m)

# Run the gradient
for i in range(epoch):
    current_M = M + alpha*regCost(Delta, P)
    grad = current_M
    M = current_M
    
    g = G(M, P)

print(g)
print(M)

TypeError: 'function' object is not subscriptable